This script was written to pull data for a project I completed on Benford's Law.  It takes all of the line items from the past 10 years of financial statements (cash flows, balance sheet, income statement) for companies listed on the Dow Jones and compiles the data into one table.

In [1]:
import requests
import pandas as pd
import numpy as np
import json
import xlsxwriter

In [2]:
#tickers = pd.read_csv('dowtickers.csv', encoding = "ISO-8859-1")
#tickers = list(tickers['Company'])
#tickers = [tick.split()[0] for tick in tickers]

tickers = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 
           'DIS', 'DOW', 'XOM', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 
           'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 
           'UTX', 'UNH', 'VZ', 'V', 'WMT', 'WBA']

In [ ]:
# Function to retrieve a statement for each of the companies (by ticker)
def retrieveFinancials(statement_type, tickers=tickers):
    statements = []
    for ticker in tickers:
        try:
            r = requests.get(
                'https://financialmodelingprep.com/api/v3/financials/'+ statement_type + '/' + ticker.upper()).json()
            statements.append(r)
        except:
            print('Request for {} failed'.format(ticker))
    return statements

# Get financial data for each ticker
cash_flows, balance_sheets, income_statements = list(
    map(retrieveFinancials, ['cash-flow-statement', 'balance-sheet-statement', 'income-statement']))

In [26]:
cash_flows[0]

{'symbol': 'MMM',
 'financials': [{'date': '2018-12-31',
   'Depreciation & Amortization': '1488000000.0',
   'Stock-based compensation': '302000000.0',
   'Operating Cash Flow': '6439000000.0',
   'Capital Expenditure': '-1315000000.0',
   'Acquisitions and disposals': '859000000.0',
   'Investment purchases and sales': '669000000.0',
   'Investing Cash flow': '222000000.0',
   'Issuance (repayment) of debt': '933000000.0',
   'Issuance (buybacks) of shares': '-4385000000.0',
   'Dividend payments': '-3193000000.0',
   'Financing Cash Flow': '-6701000000.0',
   'Effect of forex changes on cash': '-160000000.0',
   'Net cash flow / Change in cash': '-200000000.0',
   'Free Cash Flow': '5124000000.0',
   'Net Cash/Marketcap': '-0.1027'},
  {'date': '2017-12-31',
   'Depreciation & Amortization': '1544000000.0',
   'Stock-based compensation': '324000000.0',
   'Operating Cash Flow': '6240000000.0',
   'Capital Expenditure': '-1324000000.0',
   'Acquisitions and disposals': '-958000000.0'

In [39]:
# Aggregate relevant data from JSON response into one dictionary
def dataGather(statement, abbreviation):
    #tickers = [company['symbol'] for company in statement]
    data = {}
    for company in statement:
        nums = []
        tick = company['symbol']
        fin = company['financials']
        for year in fin:
            for line in [line for line in year.keys() if line != 'date']:
                try:
                    value = float(year[line])
                    if value > 100: # Prevent collection of dates
                        nums.append(value)
                except:
                    continue
            data[abbreviation + '.' + tick] = nums
    return data

cf = dataGather(cash_flows, 'cf')
bs = dataGather(balance_sheets, 'bs')
is_ = dataGather(income_statements, 'is')

dfdct = {**cf, **bs, **is_}

In [72]:
# Write data to Excel workbook
# This creates a column for each company-statement combination
# The column only contains numbers
workbook = xlsxwriter.Workbook('benford_data.xlsx') 
worksheet = workbook.add_worksheet('data')
col = 0
for key in list(dfdct.keys()):
    # Write the data if there are more than 100 items on the statement
    if len(dfdct[key]) >= 100:
        worksheet.write(0, col, key)
        row = 1
        for num in dfdct[key]:
            worksheet.write(row, col, num)
            print(row, col, num)
            row += 1
        col += 1

1 0 2853000000.0
2 0 380000000.0
3 0 3233000000.0
4 0 5020000000.0
5 0 4366000000.0
6 0 13709000000.0
7 0 8738000000.0
8 0 12708000000.0
9 0 22791000000.0
10 0 36500000000.0
11 0 2266000000.0
12 0 1211000000.0
13 0 7244000000.0
14 0 13411000000.0
15 0 14622000000.0
16 0 243000000.0
17 0 19408000000.0
18 0 26652000000.0
19 0 40636000000.0
20 0 9796000000.0
21 0 380000000.0
22 0 11389000000.0
23 0 1090000000.0
24 0 3767000000.0
25 0 3053000000.0
26 0 1076000000.0
27 0 4129000000.0
28 0 4911000000.0
29 0 4034000000.0
30 0 14277000000.0
31 0 8866000000.0
32 0 13449000000.0
33 0 23710000000.0
34 0 37987000000.0
35 0 1945000000.0
36 0 1853000000.0
37 0 7687000000.0
38 0 12096000000.0
39 0 13949000000.0
40 0 310000000.0
41 0 18678000000.0
42 0 26365000000.0
43 0 39115000000.0
44 0 11563000000.0
45 0 1076000000.0
46 0 9820000000.0
47 0 1203000000.0
48 0 3889000000.0
49 0 2398000000.0
50 0 280000000.0
51 0 2678000000.0
52 0 4392000000.0
53 0 3385000000.0
54 0 11726000000.0
55 0 8516000000.0
56 

224 2 9924000000.0
225 2 1051000000.0
226 2 41678000000.0
227 2 4768000000.0
228 2 1083000000.0
229 2 25391000000.0
230 2 1636000000.0
231 2 33505000000.0
232 2 75183000000.0
233 2 12015000000.0
234 2 20722000000.0
235 2 4123000000.0
236 2 6670000000.0
237 2 27392000000.0
238 2 37169000000.0
239 2 47791000000.0
240 2 39750000000.0
241 2 5083000000.0
242 2 8707000000.0
243 2 5263000000.0
244 2 18201000000.0
245 2 23464000000.0
246 2 5057000000.0
247 2 455000000.0
248 2 15946000000.0
249 2 2954000000.0
250 2 453000000.0
251 2 10528000000.0
252 2 1135000000.0
253 2 4355000000.0
254 2 47501000000.0
255 2 5601000000.0
256 2 11506000000.0
257 2 2906000000.0
258 2 4355000000.0
259 2 15861000000.0
260 2 77000000.0
261 2 23353000000.0
262 2 31640000000.0
263 2 28729000000.0
264 2 2579000000.0
265 2 5905000000.0
1 3 7637000000.0
2 3 927000000.0
3 3 8564000000.0
4 3 16782000000.0
5 3 62567000000.0
6 3 87830000000.0
7 3 12645000000.0
8 3 11269000000.0
9 3 1087000000.0
10 3 284000000.0
11 3 2952900

48 5 33941000000.0
49 5 7980000000.0
50 5 6988000000.0
51 5 13000000.0
52 5 7001000000.0
53 5 16577000000.0
54 5 5419000000.0
55 5 29619000000.0
56 5 182186000000.0
57 5 4581000000.0
58 5 30250000000.0
59 5 230459000000.0
60 5 260078000000.0
61 5 13986000000.0
62 5 10840000000.0
63 5 31785000000.0
64 5 35286000000.0
65 5 46126000000.0
66 5 19593000000.0
67 5 81571000000.0
68 5 113356000000.0
69 5 173046000000.0
70 5 145556000000.0
71 5 30263000000.0
72 5 39125000000.0
73 5 622000000.0
74 5 6959000000.0
75 5 11022000000.0
76 5 310000000.0
77 5 11332000000.0
78 5 15272000000.0
79 5 6334000000.0
80 5 34430000000.0
81 5 188396000000.0
82 5 4588000000.0
83 5 27110000000.0
84 5 230110000000.0
85 5 264540000000.0
86 5 13516000000.0
87 5 4927000000.0
88 5 25467000000.0
89 5 33622000000.0
90 5 38549000000.0
91 5 22356000000.0
92 5 85187000000.0
93 5 110654000000.0
94 5 181578000000.0
95 5 152716000000.0
96 5 27420000000.0
97 5 27217000000.0
98 5 1492000000.0
99 5 7024000000.0
100 5 12793000000.

194 7 5056000000.0
195 7 23770000000.0
196 7 48053000000.0
197 7 53621000000.0
198 7 31635000000.0
199 7 8518000000.0
200 7 14533000000.0
201 7 3450000000.0
202 7 362000000.0
203 7 11199000000.0
204 7 138000000.0
205 7 11337000000.0
206 7 4430000000.0
207 7 2650000000.0
208 7 21579000000.0
209 7 14727000000.0
210 7 26909000000.0
211 7 7585000000.0
212 7 51342000000.0
213 7 72921000000.0
214 7 8859000000.0
215 7 9376000000.0
216 7 18508000000.0
217 7 14041000000.0
218 7 23417000000.0
219 7 4534000000.0
220 7 23096000000.0
221 7 41604000000.0
222 7 49278000000.0
223 7 31003000000.0
224 7 7723000000.0
225 7 12080000000.0
226 7 3162000000.0
227 7 273000000.0
228 7 9151000000.0
229 7 62000000.0
230 7 9213000000.0
231 7 3758000000.0
232 7 2354000000.0
233 7 17551000000.0
234 7 9561000000.0
235 7 12828000000.0
236 7 6755000000.0
237 7 31120000000.0
238 7 48671000000.0
239 7 6657000000.0
240 7 6800000000.0
241 7 13721000000.0
242 7 5059000000.0
243 7 11859000000.0
244 7 1844000000.0
245 7 9604

122 10 57163000000.0
123 10 77356000000.0
124 10 711614000000.0
125 10 74954000000.0
126 10 67900000000.0
127 10 848942000000.0
128 10 185634000000.0
129 10 519321000000.0
130 10 39418000000.0
131 10 778228000000.0
132 10 50252000000.0
133 10 70714000000.0
134 10 676620000000.0
1 11 1778000000.0
2 11 1778000000.0
3 11 1936000000.0
4 11 13925000000.0
5 11 18529000000.0
6 11 22375000000.0
7 11 2252000000.0
8 11 25474000000.0
9 11 44003000000.0
10 11 7755000000.0
11 11 2395000000.0
12 11 16716000000.0
13 11 26807000000.0
14 11 29202000000.0
15 11 1782000000.0
16 11 656000000.0
17 11 29165000000.0
18 11 45881000000.0
19 11 46423000000.0
20 11 27424000000.0
21 11 890000000.0
22 11 6566000000.0
23 11 3595000000.0
24 11 3595000000.0
25 11 1952000000.0
26 11 12748000000.0
27 11 18933000000.0
28 11 22075000000.0
29 11 2275000000.0
30 11 25596000000.0
31 11 44529000000.0
32 11 7244000000.0
33 11 2761000000.0
34 11 16194000000.0
35 11 24267000000.0
36 11 27028000000.0
37 11 1805000000.0
38 11 101

...

124 41 22604000000.0
125 41 16604000000.0
126 41 106916000000.0
127 41 56778000000.0
128 41 50138000000.0
129 41 6258000000.0
130 41 23594000000.0
131 41 28724000000.0
132 41 21414000000.0
133 41 411000000.0
134 41 21003000000.0
135 41 5148000000.0
136 41 15855000000.0
137 41 15855000000.0
138 41 1178618168.0
139 41 1196951006.0
140 41 26229000000.0
141 41 21414000000.0
142 41 15855000000.0
143 41 99870000000.0
144 41 53857000000.0
145 41 46013000000.0
146 41 6026000000.0
147 41 21837000000.0
148 41 25922000000.0
149 41 20091000000.0
150 41 368000000.0
151 41 19723000000.0
152 41 4890000000.0
153 41 14833000000.0
154 41 14833000000.0
155 41 1242361137.0
156 41 1268789202.0
157 41 24922000000.0
158 41 20091000000.0
159 41 14833000000.0
160 41 95758000000.0
161 41 51973000000.0
162 41 43785000000.0
163 41 5820000000.0
164 41 20952000000.0
165 41 25244000000.0
166 41 18541000000.0
167 41 402000000.0
168 41 18138000000.0
169 41 4713000000.0
170 41 13425000000.0
171 41 13425000000.0
172 41 

122 45 5503100000.0
123 45 5503100000.0
124 45 1023218317.0
125 45 1032100000.0
126 45 9920000000.0
127 45 8505000000.0
128 45 5503100000.0
129 45 24074600000.0
130 45 14437300000.0
131 45 9637300000.0
132 45 2333300000.0
133 45 2164200000.0
134 45 7473100000.0
135 45 450900000.0
136 45 7000300000.0
137 45 2054000000.0
138 45 4946300000.0
139 45 4946300000.0
140 45 1056506714.0
141 45 1066000000.0
142 45 8727400000.0
143 45 7451200000.0
144 45 4946300000.0
145 45 22744700000.0
146 45 13952900000.0
147 45 8791800000.0
148 45 2234200000.0
149 45 1950800000.0
150 45 6841000000.0
151 45 473200000.0
152 45 6487000000.0
153 45 1936000000.0
154 45 4551000000.0
155 45 4551000000.0
156 45 1079186614.0
157 45 1092200000.0
158 45 8176400000.0
159 45 6960200000.0
160 45 4551000000.0
1 46 42294000000.0
2 46 13509000000.0
3 46 28785000000.0
4 46 9752000000.0
5 46 10102000000.0
6 46 20486000000.0
7 46 8299000000.0
8 46 8728000000.0
9 46 2508000000.0
10 46 6220000000.0
11 46 6220000000.0
12 46 2600376

108 49 11242000000.0
109 49 16791000000.0
110 49 2221000000.0
111 49 28000000.0
112 49 14570000000.0
113 49 14570000000.0
114 49 7362573528.0
115 49 7442000000.0
116 49 24446000000.0
117 49 16791000000.0
118 49 14598000000.0
119 49 61035000000.0
120 49 12500000000.0
121 49 48535000000.0
122 49 8681000000.0
123 49 17581000000.0
124 49 37054000000.0
125 49 11481000000.0
126 49 13630000000.0
127 49 3621000000.0
128 49 40000000.0
129 49 10009000000.0
130 49 10009000000.0
131 49 7686966786.0
132 49 7817000000.0
133 49 22656000000.0
134 49 13630000000.0
135 49 10049000000.0
136 49 65165000000.0
137 49 14788000000.0
138 49 50377000000.0
139 49 9483000000.0
140 49 18973000000.0
141 49 40906000000.0
142 49 9471000000.0
143 49 9410000000.0
144 49 1153000000.0
145 49 31000000.0
146 49 30000000.0
147 49 8257000000.0
148 49 8257000000.0
149 49 8009951287.0
150 49 8036000000.0
151 49 17809000000.0
152 49 9410000000.0
153 49 8288000000.0
154 49 49269000000.0
155 49 8459000000.0
156 49 40810000000.0
1

86 53 92550000000.0
87 53 29939000000.0
88 53 18941000000.0
89 53 20316000000.0
90 53 9623000000.0
91 53 708000000.0
92 53 8867000000.0
93 53 3242000000.0
94 53 48000000.0
95 53 5625000000.0
96 53 5625000000.0
97 53 1006758191.0
98 53 1006000000.0
99 53 10950000000.0
100 53 9575000000.0
101 53 5673000000.0
102 53 110618000000.0
103 53 82749000000.0
104 53 27869000000.0
105 53 17306000000.0
106 53 18615000000.0
107 53 9254000000.0
108 53 632000000.0
109 53 8622000000.0
110 53 3096000000.0
111 53 5526000000.0
112 53 5526000000.0
113 53 1021492625.0
114 53 1027000000.0
115 53 10563000000.0
116 53 9254000000.0
117 53 5526000000.0
118 53 101862000000.0
119 53 76717000000.0
120 53 25145000000.0
121 53 15557000000.0
122 53 16681000000.0
123 53 8464000000.0
124 53 505000000.0
125 53 7959000000.0
126 53 2817000000.0
127 53 5142000000.0
128 53 5142000000.0
129 53 1066026494.0
130 53 1070000000.0
131 53 9588000000.0
132 53 8464000000.0
133 53 5142000000.0
134 53 94155000000.0
135 53 70957000000.0

75 57 5276000000.0
76 57 1056000000.0
77 57 59000000.0
78 57 4220000000.0
79 57 4220000000.0
80 57 1092282712.0
81 57 1043200000.0
82 57 7623000000.0
83 57 5881000000.0
84 57 4279000000.0
85 57 76392000000.0
86 57 54823000000.0
87 57 21569000000.0
88 57 17992000000.0
89 57 17375000000.0
90 57 4194000000.0
91 57 156000000.0
92 57 3458000000.0
93 57 1526000000.0
94 57 99000000.0
95 57 1932000000.0
96 57 1932000000.0
97 57 956560612.0
98 57 953100000.0
99 57 4930000000.0
100 57 3614000000.0
101 57 2031000000.0
102 57 72217000000.0
103 57 51098000000.0
104 57 21119000000.0
105 57 17543000000.0
106 57 17047000000.0
107 57 4072000000.0
108 57 165000000.0
109 57 4047000000.0
110 57 1499000000.0
111 57 2548000000.0
112 57 2548000000.0
113 57 945003680.0
114 57 946000000.0
115 57 5495000000.0
116 57 4212000000.0
117 57 2548000000.0
118 57 71633000000.0
119 57 51291000000.0
120 57 20342000000.0
121 57 16878000000.0
122 57 16878000000.0
123 57 3464000000.0
124 57 88000000.0
125 57 3376000000.0
12